# CRISPR-guided multiomics: PubMed mTOR/Hippo literature analysis

## Overview

Using data from PubMed and Pubtator Central, scores are assigned to each gene based on how often they appear in articles mentioning Hippo / mTOR signaling.

**Approach**:

1. Load Pubmed public article data (~22m article titles/abstracts)
2. Find all articles containing the strings "hippo signaling" or "hippo pathway" (Hippo) and "mtor" or "target of rapamycin" (mTOR).
3. Compute the ratio of the # of gene occurrences associated with each of the query terms vs. how often the gene would be expected to appear alongside of the same number of articles if no such association is present.

**Datasets used**

- `articles-baseline-full.csv` - Pubmed article title, abstract, etc. in tabular format.
- `bioconcepts2pubtatorcentral_filtered_human_genes.feather` - Pubmed human gene annotations from PubTator Central.
- `grch38.tsv.gz` - GRCh38 gene annotations from the `annotables` R package.

**Limitations**

- PubTator Central annotations only cover a subset of PubMed articles
- Gene annotations may be located anywhere in the article text, while "mTOR" / "Hippo signaling" checks are limited to the article titles/abstracts.

**Data sources**

1. https://ftp.ncbi.nlm.nih.gov/pubmed/baseline/
2. https://ftp.ncbi.nlm.nih.gov/pub/lu/PubTatorCentral/
3. https://github.com/stephenturner/annotables

## Methods

### Setup

In [2]:
import os
import pandas as pd
from typing import List

out_dir = "output"

if not os.path.exists(out_dir):
    os.makedirs(out_dir, mode=0o755)

Below, PubTator Central gene annotations are loaded from a filtered version of the dataset which includes only genes from articles which are also annotated as mentioning "human".

This is donely purely for convenience / speed up the process of working with the data.

In [3]:
# load pubtator article x gene count data
pubtator_genes = pd.read_feather("data/bioconcepts2pubtatorcentral_filtered_human_genes.feather")

pubtator_genes = pubtator_genes.astype({"pmid": "int",
                                        "concept_id": "int", 
                                        "type": "category", 
                                        "mentions": "string", 
                                        "resource": "category"})

pubtator_genes.shape

(31528536, 5)

In [4]:
# preview of pubtator dataframe
pubtator_genes.head()

,pmid,type,concept_id,mentions,resource
0,23574000,Gene,3497,IgE,GNormPlus
1,3921000,Gene,140738,PR,GNormPlus
2,17505000,Gene,1113,chromogranin A,GNormPlus
3,17505000,Gene,4255,O(6)-methylguanine DNA methyltransferase,GNormPlus
4,21677000,Gene,64399,hip,GNormPlus


In [5]:
# number of unique pubmed articles
total_articles = len(set(pubtator_genes.pmid))
total_articles

4880846

Below, PubMed article titles and abstracts from the PubMed public article data are loaded.
These are used to check for mentions of the Hippo signaling pathway / mTOR.

The csv file simply stores the raw titles/abstracts, as extracted from the source XML files.

In [6]:
# load pubmed title/abstract data (~22.7m articles/rows)
infile = "data/articles-baseline-full.csv"

pubmed = pd.read_csv(infile, dtype={"id": "int",
                                    "doi": "string", 
                                    "title": "string",
                                    "abstract": "string", 
                                    "date": "string"})

pubmed.shape

(22754173, 5)

In [7]:
# exclude articles for which no human genes are annotated in pubtator
# (helps to reduce memory footprint; ~22.7 -> 4.6m rows)
pmids = set(pubtator_genes.pmid)
mask = pubmed.id.isin(pmids)

mask.value_counts()

pubmed = pubmed[mask]

In [8]:
# convert titles/abstracts to lowercase and combine
pubmed['abstract'] = pubmed['abstract'].str.lower()
pubmed['title'] = pubmed['title'].str.lower()

# combine title & text into a single string (performing piecewise, to reduce memory requirements)
pubmed['text'] = pubmed['title'] + " "
pubmed['text'] = pubmed['text'] + pubmed['abstract']

pubmed.drop(columns=['title', 'abstract'], inplace=True)

In [9]:
# load gene annotations (source: Annotables R package)
annot = pd.read_csv("data/grch38.tsv.gz", sep='\t')

In [10]:
# generate a table of total gene counts across all articles
global_counts = pubtator_genes.concept_id.value_counts().sort_values(ascending=False)
global_counts = global_counts.to_frame().rename(columns={"concept_id": "num_articles_global"})

# drop genes that can't be mapped from entrez ids -> symbols
mask = global_counts.index.isin(annot.entrez)
global_counts = global_counts[mask]

# add gene symbols
gene_symbols = annot.set_index('entrez').loc[global_counts.index].symbol
gene_symbols = gene_symbols[~gene_symbols.to_frame().index.duplicated(keep='first')]
global_counts.index = gene_symbols

# combine counts for duplicated symbols
global_counts = global_counts.groupby('symbol').sum()

global_counts.head()

,num_articles_global
symbol,
A1BG,3652
A1BG-AS1,30
A1CF,172
A2M,9406
A2M-AS1,32


### helper functions

In [11]:
def pmids_to_genes(pmids: List[str]) -> pd.DataFrame:
    """
    Helper function to return a table of raw and adjusted gene counts associated with a given set of
    pubmed ids.
    
    Since some genes are discussed much more frequently in the literature than others, an "adjusted" 
    count is computed which modifies the raw counts based on the expected likelihood of a similar number of
    mentions ocurring for the gene simply due to chance.    
    
    Arguments
    ---------
    pmids: List[str]
        List of PubMed article IDs
        
    Return
    ------
    pd.DataFrame
        DataFrame containing counts of each gene mentioned at least once in titles/abstracts of the query articles.
    """
    gene_counts = pubtator_genes[pubtator_genes.pmid.isin(pmids)].concept_id.value_counts()

    # drop genes that can't be mapped from entrez ids -> symbols
    mask = gene_counts.index.isin(annot.entrez)
    gene_counts = gene_counts[mask]

    gene_counts = gene_counts.to_frame().rename(columns={"concept_id": "num_articles_query"})

    gene_symbols = annot.set_index('entrez').loc[gene_counts.index].symbol
    
    # drop duplicated indices
    gene_symbols = gene_symbols[~gene_symbols.to_frame().index.duplicated(keep='first')]

    gene_counts.index = gene_symbols
    
    # expected gene count = <average gene mentions / article> * <num articles in batch>
    # (later, a better model of expected gene counts for a article batch of size "N" could be employed..)
    expected_counts = global_counts.copy()
    expected_counts['num_articles_expected'] = (expected_counts['num_articles_global'] / total_articles) * len(pmids)

    # add a "score" column reflecting the ratio of observed/expected gene mentions
    gene_counts = pd.merge(gene_counts, expected_counts, left_index=True, right_index=True)
    gene_counts['score'] = gene_counts.num_articles_query / gene_counts.num_articles_expected

    gene_counts = gene_counts.sort_values('score', ascending=False)

    return gene_counts

### Find articles pertaining to mTOR/Hippo signaling

In [12]:
# Hippo signaling
hippo_mask1 = pubmed.text.str.contains('hippo signaling')
hippo_mask2 = pubmed.text.str.contains('hippo pathway')
hippo_mask = hippo_mask1 | hippo_mask2

print(f"- # Hippo articles: {hippo_mask.sum()}")

- # Hippo articles: 1911


In [13]:
# mTOR
mtor_mask1 = pubmed.text.str.contains(r'\bmtor\b')
mtor_mask2 = pubmed.text.str.contains('target of rapamycin')
mtor_mask = mtor_mask1 | mtor_mask2

print(f"- # mTOR articles: {mtor_mask.sum()}")

- # mTOR articles: 21600


In [14]:
# articles mentioning both Hippo and mTOR?
print(f"- # Hippo/mTOR articles: {(hippo_mask & mtor_mask).sum()}")

- # Hippo/mTOR articles: 44


In [15]:
# Gene rankings for Hippo, mTOR, or both
hippo_genes = pmids_to_genes(pubmed[hippo_mask].id.values.tolist())
mtor_genes = pmids_to_genes(pubmed[mtor_mask].id.values.tolist())
mtor_hippo_genes = pmids_to_genes(pubmed[hippo_mask & mtor_mask].id.values.tolist())

# exclude genes only with only a single association
hippo_genes = hippo_genes[hippo_genes.num_articles_query >= 2]
mtor_genes = mtor_genes[mtor_genes.num_articles_query >= 2]
mtor_hippo_genes = mtor_hippo_genes[mtor_hippo_genes.num_articles_query >= 2]

### Genes commonly associated with Hippo

In [16]:
hippo_genes.head(50)

,num_articles_query,num_articles_global,num_articles_expected,score
symbol,,,,
MIR8070,2,2,0.000783,2554.079540
MOB1B,103,226,0.088486,1164.027401
MOB1A,307,739,0.289341,1061.031690
SAV1,382,966,0.378218,1009.998327
VGLL4,126,365,0.142909,881.682252
STK3,484,1572,0.615486,786.370545
LINC01048,4,13,0.005090,785.870628
LATS1,675,2262,0.885642,762.159014
WWC3,21,82,0.032105,654.093541


### Genes commonly associated with mTOR

In [17]:
mtor_genes.head(50)

,num_articles_query,num_articles_global,num_articles_expected,score
symbol,,,,
C12orf66,21,33,0.146040,143.795968
PRR5,195,336,1.486955,131.140456
WDR59,40,71,0.314208,127.304278
WDR24,41,73,0.323059,126.911901
RRAGB,165,325,1.438275,114.720739
MLST8,587,1244,5.505275,106.625008
DEPTOR,610,1295,5.730974,106.439156
ITFG2,22,48,0.212422,103.567334
CASTOR1,60,135,0.597437,100.428930


### Genes commonly associated with both Hippo + mTOR

In [18]:
mtor_hippo_genes.head(50)

,num_articles_query,num_articles_global,num_articles_expected,score
symbol,,,,
VGLL4,4,365,0.003290,1215.652802
MOB1B,2,226,0.002037,981.666533
MOB1A,6,739,0.006662,900.635872
SAV1,7,966,0.008708,803.828393
AMOT,4,658,0.005932,674.336281
AMOTL2,2,351,0.003164,632.070189
LATS1,12,2262,0.020392,588.479142
STK3,8,1572,0.014171,564.520703
TEAD1,6,1492,0.013450,446.092432


### Create combined stats table

In [19]:
# convert gene counts to ranks
mtor_gene_ranks = mtor_genes.rank(axis=0, ascending=False) 
hippo_gene_ranks = hippo_genes.rank(axis=0, ascending=False) 

# create combined <gene x topic> co-occurrence stats table
drop_cols = ['num_articles_expected', 'num_articles_global']
x1 = mtor_genes.reset_index().drop(columns=drop_cols).rename(columns={'num_articles_query': 'mtor_num_articles', 'score': 'mtor_association_score'})
x2 = hippo_genes.reset_index().drop(columns=drop_cols).rename(columns={'num_articles_query': 'hippo_num_articles', 'score': 'hippo_association_score'})
x3 = mtor_hippo_genes.reset_index().drop(columns=drop_cols).rename(columns={'num_articles_query': 'mtor_plus_hippo_num_articles', 'score': 'mtor_plus_hippo_association_score'})

stats = x1.merge(x2, on='symbol', how='outer').merge(x3, on='symbol', how='outer')
stats = stats.fillna(0)

# add total article/score count across all queries
stats['total_articles'] = stats.mtor_num_articles + stats.hippo_num_articles + stats.mtor_plus_hippo_num_articles
stats['total_score'] = stats.mtor_association_score + stats.hippo_association_score + stats.mtor_plus_hippo_association_score
stats = stats.sort_values('total_score', ascending=False)

# add global article count
stats.insert(1, "num_articles_global", global_counts.loc[stats.symbol].values)

stats.reset_index(drop=True)

,symbol,num_articles_global,mtor_num_articles,mtor_association_score,hippo_num_articles,hippo_association_score,mtor_plus_hippo_num_articles,mtor_plus_hippo_association_score,total_articles,total_score
0,MIR8070,2,0.0,0.000000,2.0,2554.079540,0.0,0.000000,2.0,2554.079540
1,MOB1B,226,4.0,3.999382,103.0,1164.027401,2.0,981.666533,109.0,2149.693316
2,VGLL4,365,10.0,6.190824,126.0,881.682252,4.0,1215.652802,140.0,2103.525878
3,MOB1A,739,15.0,4.586572,307.0,1061.031690,6.0,900.635872,328.0,1966.254133
4,SAV1,966,15.0,3.508775,382.0,1009.998327,7.0,803.828393,404.0,1817.335495
...,...,...,...,...,...,...,...,...,...,...
9506,HBA1,22482,10.0,0.100509,0.0,0.000000,0.0,0.000000,10.0,0.100509
9507,ZFPM1,6874,3.0,0.098617,0.0,0.000000,0.0,0.000000,3.0,0.098617
9508,SRI,20834,9.0,0.097614,0.0,0.000000,0.0,0.000000,9.0,0.097614
9509,HHIP,62935,16.0,0.057447,0.0,0.000000,0.0,0.000000,16.0,0.057447


### Save results

In [20]:
mtor_hippo_genes.to_csv(os.path.join(out_dir, 'hippo_plus_mtor_article_gene_stats.csv'))
hippo_genes.to_csv(os.path.join(out_dir, 'hippo_article_gene_stats.csv'))
mtor_genes.to_csv(os.path.join(out_dir, 'mtor_article_gene_stats.csv'))

stats.to_csv(os.path.join(out_dir, 'gene_topic_co-occurrence_stats.csv'), index=False)